# Trabajo Final Parte 2

In [ ]:
# Instalación de dependencias para Google Colab
!pip install deltalake
!pip install pyarrow
!pip install pandas
!pip install requests
!pip install numpy

In [1]:
import os, time, requests
import pandas as pd
from deltalake import write_deltalake, DeltaTable
from datetime import datetime
import numpy as np

## Funciones de carga de datos desde Bronze

In [2]:
def load_bronze_coin_list():
    """
    carga la lista de monedas desde el Delta Lake.
    El df tiene buen formato y no requiere limpieza.
    retorna: un DataFrame de pandas con la lista de monedas disponibles en la Api.
    """
    path="bronze/coingecko_api/coins_list"
    try:
        table=DeltaTable(path)
        arrow_table=table.to_pyarrow_table()
        df= arrow_table.to_pandas()
        
        return df
    except FileNotFoundError:
        raise FileNotFoundError(f"Ruta no encontrada: {path}")
    except Exception as e:
        # Captura cualquier otro error (DeltaCorruption, permisos, etc.)
        raise RuntimeError(f"Error cargando datos de Delta Lake en {path}: {e}")

def load_bronze_coins_markets_latest():
    """
    Carga solo los datos más recientes del endpoint coins/markets
    retorna: un DataFrame de pandas con los datos más recientes del mercado de criptomonedas.
    El dataframe contiene 31 columnas y valores nulos.
    """
    path = "bronze/coingecko_api/coins_markets"
    try:
        # Cargar toda la tabla
        table = DeltaTable(path)
        df = table.to_pyarrow_table().to_pandas()
        
        # Obtener la fecha más reciente
        latest_date = df['extract_date'].max()
        
        # Filtrar solo esa fecha
        df_latest = df[df['extract_date'] == latest_date].copy()
        
        print(f"📅 Fecha más reciente: {latest_date}")
        print(f"📊 Registros de la última extracción: {len(df_latest)}")
        
        return df_latest
    except FileNotFoundError:
        raise FileNotFoundError(f"Ruta no encontrada: {path}")
    except Exception as e:
        # Captura cualquier otro error (DeltaCorruption, permisos, etc.)
        raise RuntimeError(f"Error cargando datos de Delta Lake en {path}: {e}")



def load_bronze_market_chart_latest():
    """
    Carga solo los datos más recientes del endpoint coins/markets
    retorna: un DataFrame de pandas con los datos de los ultimos 30 dias de las 5 mejores monedas del mercado de criptomonedas, toma el dataframe almacenado mas reciente
    el dataframe tiene columnas: ['prices', 'market_caps', 'total_volumes', 'extract_date','coin_id'] las 3 primeras columns tienen una lista de listas con dos elementos: [timestamp, valor].
    """
    path = "bronze/coingecko_api/market_chart"
    try:
        # Cargar toda la tabla
        table = DeltaTable(path)
        df = table.to_pyarrow_table().to_pandas()
        
        # Obtener la fecha más reciente
        latest_date = df['extract_date'].max()
        
        # Filtrar solo esa fecha
        df_latest = df[df['extract_date'] == latest_date].copy()
        
        print(f"📅 Fecha más reciente: {latest_date}")
        print(f"📊 Registros de la última extracción: {len(df_latest)}")
        
        return df_latest
    except FileNotFoundError:
        raise FileNotFoundError(f"Ruta no encontrada: {path}")
    except Exception as e:
        # Captura cualquier otro error (DeltaCorruption, permisos, etc.)
        raise RuntimeError(f"Error cargando datos de Delta Lake en {path}: {e}")


# Funciones de procesamiento y transformacion

In [ ]:

def df_market_select_columns(df_market):
    df_market_clean = df_market[["id", "symbol", "name","extract_date", "current_price", "market_cap", "total_volume", "high_24h", "low_24h", "price_change_24h", "price_change_percentage_24h", "market_cap_change_24h", "market_cap_change_percentage_24h", "total_supply", "max_supply", "ath", "ath_date", "roi.percentage"]].copy()
    
    return df_market_clean

def df_market_fill_nulls(df):
    # imputar valores nulos
    # columna max_supply, los valores nulos se deben a que hay monedas que no tienen un suministro máximo definidos
    # usaremos la mediana como valor por defecto asi no se  introducen outliers extremos y se Conserva la escala de magnitudes.
    df_market = df.copy()
    median_max = df_market['max_supply'].median()
    df_market['max_supply'] = df_market['max_supply'].fillna(median_max)

    #columna roi.percentage, los valores nulos se deben a que hay monedas son nuevas y no tienen un retorno de inversión definido usaremos 0 como valor por defecto
    df_market['roi.percentage']=df_market['roi.percentage'].fillna(0)
    
    return df_market

def transform_coin_market_data(df_market):
    """
    """
    try:
        df_market_columns=df_market_select_columns(df_market)
        df_market_clean=df_market_fill_nulls(df_market_columns)
        return df_market_clean
    except Exception as e:
        raise RuntimeError(f"Error transormando los datos de df_market: {e}") 

def transform_market_chart_data(df_market_chart):
    """
    aplica 3 transformacion importantes al df_market_chart:
        - se crea una columna date extrayendo el valor timestamp de cada tupla
        - las columnas prices, market_caps, total_volumes originalmente de tuplas se convierten en flotantes redondeados en 2 decimales extrayendo su valor de las tuplas
        - se reordenan las columnas del data frame
    retorna: df_market_chart limpio.
    """
    df_clean=df_market_chart.copy()
    
    df_clean['date']=pd.to_datetime(df_clean['prices'].apply(lambda x: x[0]), unit='ms')# se extrae el timestamp de cada tupla y se convierte a datetime
    df_clean['prices']=df_clean['prices'].apply(lambda x: round(float(x[1]),2)) # se extrae el valor de la tupla y se convierte a float redondeado en 2 decimales
    df_clean['market_caps'] = df_clean['market_caps'].apply(lambda x: round(float(x[1]),2))
    df_clean['total_volumes']=df_clean['total_volumes'].apply(lambda x: round(float(x[1]),2))
    
    df_final = df_clean[[
    'coin_id',
    'date',
    'extract_date',
    'prices',
    'market_caps',
    'total_volumes']].reset_index(drop=True)
    return df_final
    
    


# Funciones almacenamiento en capa silver

In [ ]:
SILVER_PATH="silver/coingecko_api"


def ensure_directory_exists(path):
    """
    Asegura que el directorio existe, si no lo crea automáticamente
    Compatible con Google Colab
    """
    try:
        if not os.path.exists(path):
            os.makedirs(path, exist_ok=True)
            print(f"✓ Directorio creado: {path}")
        else:
            print(f"✓ Directorio ya existe: {path}")
    except Exception as e:
        print(f"Error creando directorio {path}: {e}")
        raise

def is_valid_delta_table(path):
    """
    Verifica si un path contiene una tabla Delta válida
    """
    try:
        DeltaTable(path)
        return True
    except Exception:
        return False


def save_static_data_silver(df_static):
    """
    Guarda datos estáticos en la capa Bronze
    Estructura: silver/coingecko_api/coins_list/
    """
    path = f"{SILVER_PATH}/coins_list"
    ensure_directory_exists(path)

    print(f"💾 Guardando datos estáticos en: {path}")
    try:
        write_deltalake(
            path,
            df_static,
            mode="overwrite"
        )
        print(f"✅ Datos estáticos guardados exitosamente: {len(df_static)} registros")

        # Verificar que se guardó correctamente
        if os.path.exists(path):
            print(f"✅ Verificación: Directorio Delta creado correctamente")

    except Exception as e:
        print(f"❌ Error guardando datos estáticos: {e}")
        raise


def save_market_data_silver(df_market):
    """
    Guarda datos temporales en la capa Bronze con estrategia de merge
    Estructura: silver/coingecko_api/coins_markets/extract_date=YYYY-MM-DD/

    Estrategia de merge:
    - Merge key: coin_id + extract_date (una versión por moneda por día)
    - Si existe: actualiza el registro (útil para re-ejecuciones)
    - Si no existe: inserta nuevo registro
    - Mantiene historial completo de todas las fechas
    """
    path = f"{SILVER_PATH}/coins_markets"
    ensure_directory_exists(path)

    extract_date = df_market["extract_date"].iloc[0]
    print(f"💾 Guardando datos de mercado en: {path}")

    try:
        # Verificar si la tabla Delta ya existe
        if is_valid_delta_table(path):
            print(f"   📋 Tabla Delta existente detectada, ejecutando merge...")

            # Cargar la tabla Delta existente
            dt = DeltaTable(path)

            # Ejecutar merge usando id + extract_date como clave
            (
                dt.merge(
                    df_market,
                    predicate="target.id = source.id AND target.extract_date = source.extract_date",
                    source_alias="source",
                    target_alias="target"
                )
                .when_matched_update_all()  # Actualiza todos los campos si encuentra match
                .when_not_matched_insert_all()  # Inserta si no encuentra match
                .execute()
            )

            print(f"✅ Merge completado exitosamente")

        else:
            print(f"   🆕 Primera ejecución, creando tabla Delta...")
            # Primera vez, crear la tabla con overwrite
            write_deltalake(
                path,
                df_market,
                mode="overwrite",
                partition_by=["extract_date"]
            )
            print(f"✅ Tabla Delta creada por primera vez: {len(df_market)} registros")

        # Verificar partición creada
        partition_path = f"{path}/extract_date={extract_date}"
        if os.path.exists(partition_path):
            print(f"✅ Verificación: Partición creada/actualizada en {partition_path}")

        # Información final
        print(f"   🎯 Registros procesados en esta ejecución: {len(df_market)}")

    except Exception as e:
        print(f"❌ Error en merge de datos de mercado: {e}")
        raise


def save_multiple_market_chart_silver(df_chart):
    """
    Guarda datos de market chart en la capa silver con estrategia incremental
    Estructura: bronze/coingecko_api/market_chart/extract_date=YYYY-MM-DD/
    
    Merge usando date + coin_id como clave natural.
    
    PROS:
    - Sin duplicación: cada (date, coin_id) aparece una sola vez
    - Storage eficiente
    - Queries más simples y rápidas
    - Datos siempre actualizados con la última extracción
    """
    path = f"{SILVER_PATH}/market_chart"
    ensure_directory_exists(path)
    
    print(f"💾 Guardando datos de market chart en: {path}")
    try:
        # Verificar si ya existe la tabla
        if is_valid_delta_table(path):
            # Cargar la tabla Delta existente
            dt = DeltaTable(path)

            # Ejecutar merge usando id + extract_date como clave
            (
                dt.merge(
                    df_chart,
                    predicate="target.coin_id = source.coin_id AND target.date = source.date",
                    source_alias="source",
                    target_alias="target"
                )
                .when_matched_update_all()  # Actualiza todos los campos si encuentra match
                .when_not_matched_insert_all()  # Inserta si no encuentra match
                .execute()
            )

            print(f"✅ Merge completado exitosamente")
        else:
            # Partición por extract_date
            write_deltalake(
                path, 
                df_chart, 
                mode="overwrite", 
                partition_by=["extract_date"]
            )
    except Exception as e:
        print(f"❌ Error guardando datos de market chart: {e}")
        raise


def silver_load_df_static():
    """
    Carga la lista de monedas desde el Delta Lake y la guarda en la capa Silver.
    """
    try:
        df_static = load_bronze_coin_list()
        print(f"📊 Registros cargados: {len(df_static)}")
        
        # Guardar en Silver
        save_static_data_silver(df_static)
        
        return df_static
    except Exception as e:
        print(f"❌ Error cargando datos estáticos: {e}")
        raise
    
def silver_etl_df_market():
    """
    Carga los datos más recientes del mercado de criptomonedas desde el Delta Lake y los transforma.
    Luego guarda los datos transformados en la capa Silver.
    """
    try:
        #extraer los datos del mercado
        df_market = load_bronze_coins_markets_latest()
        print(f"📊 Registros cargados: {len(df_market)}")
        
        # Transformar los datos
        df_market_clean = transform_coin_market_data(df_market)
        
        # Guardar en Silver
        save_market_data_silver(df_market_clean)
        
        return df_market_clean
    except Exception as e:
        print(f"❌ Error cargando datos de mercado: {e}")
        raise

def silver_etl_df_market_chart():
    """
    Carga los datos más recientes del market chart desde el Delta Lake y los transforma.
    Luego guarda los datos transformados en la capa Silver.
    """
    try:
        # Extraer los datos del market chart
        df_market_chart = load_bronze_market_chart_latest()
        print(f"📊 Registros cargados: {len(df_market_chart)}")
        
        # Transformar los datos
        df_market_chart_clean = transform_market_chart_data(df_market_chart)
        
        # Guardar en Silver
        save_multiple_market_chart_silver(df_market_chart_clean)
        
        return df_market_chart_clean
    except Exception as e:
        print(f"❌ Error cargando datos de market chart: {e}")
        raise
    



In [7]:
def main():
    """
    Función principal que ejecuta el proceso de ETL para la capa Silver.
    """
    try:
        print("🔄 Iniciando proceso de ETL para la capa Silver...")
        
        # Cargar y guardar datos estáticos
        df_static = silver_load_df_static()
        
        # Cargar y guardar datos de mercado
        df_market = silver_etl_df_market()
        
        # Cargar y guardar datos de market chart
        df_market_chart = silver_etl_df_market_chart()
        
        print("✅ Proceso de ETL completado exitosamente.")
        
    except Exception as e:
        print(f"❌ Error en el proceso de ETL: {e}")

In [16]:
if __name__ == "__main__":
    try:
        main()
        print ("🚀 Script ejecutado correctamente.")
    except Exception as e:
        print(f"❌ Error al ejecutar el script: {e}")
        raise


🔄 Iniciando proceso de ETL para la capa Silver...
📊 Registros cargados: 17507
✓ Directorio ya existe: silver/coingecko_api/coins_list
💾 Guardando datos estáticos en: silver/coingecko_api/coins_list
✅ Datos estáticos guardados exitosamente: 17507 registros
✅ Verificación: Directorio Delta creado correctamente
📅 Fecha más reciente: 2025-06-20
📊 Registros de la última extracción: 50
📊 Registros cargados: 50
✓ Directorio ya existe: silver/coingecko_api/coins_markets
💾 Guardando datos de mercado en: silver/coingecko_api/coins_markets
   📋 Tabla Delta existente detectada, ejecutando merge...
✅ Merge completado exitosamente
✅ Verificación: Partición creada/actualizada en silver/coingecko_api/coins_markets/extract_date=2025-06-20
   🎯 Registros procesados en esta ejecución: 50
📅 Fecha más reciente: 2025-06-20
📊 Registros de la última extracción: 155
📊 Registros cargados: 155
✓ Directorio ya existe: silver/coingecko_api/market_chart
💾 Guardando datos de market chart en: silver/coingecko_api/mar

## Ejemplos de las transformaciones

In [9]:
df_market=load_bronze_coins_markets_latest()
df_market.head(15)

📅 Fecha más reciente: 2025-06-20
📊 Registros de la última extracción: 50


,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,atl,atl_change_percentage,atl_date,roi,last_updated,roi.times,roi.currency,roi.percentage,extract_date,extraction_timestamp
0,bitcoin,btc,Bitcoin,https://coin-images.coingecko.com/coins/images...,103284.000000,2053198116562,1,2053203899778,3.345854e+10,106450.000000,...,67.810000,1.521989e+05,2013-07-06T00:00:00.000Z,NaN,2025-06-20T23:41:56.558Z,NaN,None,NaN,2025-06-20,2025-06-20T23:42:17.199938
1,ethereum,eth,Ethereum,https://coin-images.coingecko.com/coins/images...,2402.530000,289963194186,2,289963194186,2.044097e+10,2564.480000,...,0.432979,5.546979e+05,2015-10-20T00:00:00.000Z,NaN,2025-06-20T23:41:56.473Z,30.099051,btc,3009.905070,2025-06-20,2025-06-20T23:42:17.199938
2,tether,usdt,Tether,https://coin-images.coingecko.com/coins/images...,1.000000,155943115215,3,155943115215,2.943832e+10,1.000000,...,0.572521,7.466828e+01,2015-03-02T00:00:00.000Z,NaN,2025-06-20T23:41:58.219Z,NaN,None,NaN,2025-06-20,2025-06-20T23:42:17.199938
3,ripple,xrp,XRP,https://coin-images.coingecko.com/coins/images...,2.120000,124853928160,4,211807464460,2.242240e+09,2.180000,...,0.002686,7.877262e+04,2014-05-22T00:00:00.000Z,NaN,2025-06-20T23:41:56.139Z,NaN,None,NaN,2025-06-20,2025-06-20T23:42:17.199938
4,binancecoin,bnb,BNB,https://coin-images.coingecko.com/coins/images...,641.120000,93502694268,5,93502694268,7.742529e+08,650.780000,...,0.039818,1.609525e+06,2017-10-19T00:00:00.000Z,NaN,2025-06-20T23:41:56.541Z,NaN,None,NaN,2025-06-20,2025-06-20T23:42:17.199938
5,solana,sol,Solana,https://coin-images.coingecko.com/coins/images...,139.520000,74047117423,6,84153594120,4.939952e+09,148.590000,...,0.500801,2.767576e+04,2020-05-11T19:35:23.449Z,NaN,2025-06-20T23:41:57.120Z,NaN,None,NaN,2025-06-20,2025-06-20T23:42:17.199938
6,usd-coin,usdc,USDC,https://coin-images.coingecko.com/coins/images...,0.999706,61248492601,7,61253508777,5.969033e+09,0.999900,...,0.877647,1.390762e+01,2023-03-11T08:02:13.981Z,NaN,2025-06-20T23:41:55.933Z,NaN,None,NaN,2025-06-20,2025-06-20T23:42:17.199938
7,tron,trx,TRON,https://coin-images.coingecko.com/coins/images...,0.272720,25877583807,8,25877597419,6.300792e+08,0.275224,...,0.001804,1.502462e+04,2017-11-12T00:00:00.000Z,NaN,2025-06-20T23:41:56.130Z,142.536782,usd,14253.678232,2025-06-20,2025-06-20T23:42:17.199938
8,dogecoin,doge,Dogecoin,https://coin-images.coingecko.com/coins/images...,0.162143,24274947684,9,24278053552,1.236632e+09,0.171690,...,0.000087,1.864685e+05,2015-05-06T00:00:00.000Z,NaN,2025-06-20T23:41:55.836Z,NaN,None,NaN,2025-06-20,2025-06-20T23:42:17.199938
9,staked-ether,steth,Lido Staked Ether,https://coin-images.coingecko.com/coins/images...,2402.540000,21982398342,10,21982398342,1.313348e+07,2563.890000,...,482.900000,3.952434e+02,2020-12-22T04:08:21.854Z,NaN,2025-06-20T23:41:55.795Z,NaN,None,NaN,2025-06-20,2025-06-20T23:42:17.199938


In [10]:
# df_market tiene 31 columnas y valores nulos
df_transform=transform_coin_market_data(df_market)
df_transform.info()
df_transform.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 18 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   id                                50 non-null     object 
 1   symbol                            50 non-null     object 
 2   name                              50 non-null     object 
 3   extract_date                      50 non-null     object 
 4   current_price                     50 non-null     float64
 5   market_cap                        50 non-null     int64  
 6   total_volume                      50 non-null     float64
 7   high_24h                          50 non-null     float64
 8   low_24h                           50 non-null     float64
 9   price_change_24h                  50 non-null     float64
 10  price_change_percentage_24h       50 non-null     float64
 11  market_cap_change_24h             50 non-null     float64
 12  market_cap

id                                  0
symbol                              0
name                                0
extract_date                        0
current_price                       0
market_cap                          0
total_volume                        0
high_24h                            0
low_24h                             0
price_change_24h                    0
price_change_percentage_24h         0
market_cap_change_24h               0
market_cap_change_percentage_24h    0
total_supply                        0
max_supply                          0
ath                                 0
ath_date                            0
roi.percentage                      0
dtype: int64

In [11]:
df_market_chart=load_bronze_market_chart_latest()
df_market_chart.head(15)

📅 Fecha más reciente: 2025-06-20
📊 Registros de la última extracción: 155


,prices,market_caps,total_volumes,extract_date,coin_id
0,"[1747872000000.0, 109665.86371625263]","[1747872000000.0, 2178838967665.505]","[1747872000000.0, 60722883113.84]",2025-06-20,bitcoin
1,"[1747958400000.0, 111560.356938144]","[1747958400000.0, 2214712145787.249]","[1747958400000.0, 52218408239.45193]",2025-06-20,bitcoin
2,"[1748044800000.0, 107216.66856870624]","[1748044800000.0, 2131595896624.4407]","[1748044800000.0, 49251745837.96465]",2025-06-20,bitcoin
3,"[1748131200000.0, 107831.36374380375]","[1748131200000.0, 2142540077685.2197]","[1748131200000.0, 27128230116.517162]",2025-06-20,bitcoin
4,"[1748217600000.0, 108861.81037744327]","[1748217600000.0, 2163072841533.6133]","[1748217600000.0, 29652659665.048733]",2025-06-20,bitcoin
5,"[1748304000000.0, 109377.71513263129]","[1748304000000.0, 2173059855839.4187]","[1748304000000.0, 27950577137.918888]",2025-06-20,bitcoin
6,"[1748390400000.0, 109068.45694901445]","[1748390400000.0, 2167551887130.0073]","[1748390400000.0, 38978783508.88622]",2025-06-20,bitcoin
7,"[1748476800000.0, 107838.18431100152]","[1748476800000.0, 2143342706888.307]","[1748476800000.0, 29794278037.284946]",2025-06-20,bitcoin
8,"[1748563200000.0, 105745.41660358038]","[1748563200000.0, 2101348752694.3137]","[1748563200000.0, 38279231892.56757]",2025-06-20,bitcoin
9,"[1748649600000.0, 104010.91956242644]","[1748649600000.0, 2066830946978.0637]","[1748649600000.0, 39680060337.610214]",2025-06-20,bitcoin


In [13]:
df_market_chart=transform_market_chart_data(df_market_chart)


In [14]:
df_market_chart.head()


,coin_id,date,extract_date,prices,market_caps,total_volumes
0,bitcoin,2025-05-22,2025-06-20,109665.86,2.178839e+12,6.072288e+10
1,bitcoin,2025-05-23,2025-06-20,111560.36,2.214712e+12,5.221841e+10
2,bitcoin,2025-05-24,2025-06-20,107216.67,2.131596e+12,4.925175e+10
3,bitcoin,2025-05-25,2025-06-20,107831.36,2.142540e+12,2.712823e+10
4,bitcoin,2025-05-26,2025-06-20,108861.81,2.163073e+12,2.965266e+10


In [15]:
df_market_chart.info()
df_market_chart.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155 entries, 0 to 154
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   coin_id        155 non-null    object        
 1   date           155 non-null    datetime64[ns]
 2   extract_date   155 non-null    object        
 3   prices         155 non-null    float64       
 4   market_caps    155 non-null    float64       
 5   total_volumes  155 non-null    float64       
dtypes: datetime64[ns](1), float64(3), object(2)
memory usage: 7.4+ KB


coin_id          0
date             0
extract_date     0
prices           0
market_caps      0
total_volumes    0
dtype: int64